In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from modules.test import test as tt
import cv2
# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#GPU 1번만 사용하도록 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[1], True)
    except RuntimeError as e:
        print(e)

2023-06-30 23:43:27.888786: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-30 23:43:27.933006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def make_gradcam_heatmap(img_array, grad_model):
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        class_channel = preds[:, 0]
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)


    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy(), preds.numpy()[0][0]

def save_and_display_gradcam(img, heatmap, cam_path, alpha=0.01):
    # Load the original image
    img = keras.utils.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = plt.colormaps.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)
    
    
    #img = cv2.cvtColor(np.float32(img), cv2.COLOR_GRAY2RGB)
    #img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    #concat_img = np.concatenate((img, superimposed_img), axis=1)
    #cv2.imwrite(cam_path, concat_img)

In [3]:
model = tf.keras.models.load_model("/app/models/single_channel_single_validation_CNN_weld_only_normalization_CNN.h5")


2023-06-30 23:43:30.739120: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21793 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


In [4]:
model_layer_list = [layer.name for layer in model.layers]
last_conv_layer = model_layer_list[-5]
grad_model = tf.keras.models.Model([model.inputs], 
                                   [model.get_layer(last_conv_layer).output, model.output])

In [5]:
test_image_list, test_image_label, test_image_path = tt.load_test("/app/data/User/image/test", (512,512,1), weld_type = True)

In [7]:
for image, path in zip(test_image_list, test_image_path):
    
    image_exp = np.expand_dims(image, axis=0)
    file_name = path.split("/")[-1].split(".")[0]
    heat_map, pred = make_gradcam_heatmap(image_exp, grad_model)
    save_and_display_gradcam(image, heat_map, f"/app/temp/CAM/{file_name}_{pred}.png")
    
    
        
    